# TOPIC MODELLING

# Latent Dirichlet Allocation #

This notebook is a part of my learning journey which I've been documenting from Udacity's Natural Language Processing Nanodegree program, which helped me a lot to learn and excel advanced data science stuff such as PySpark. Thank you so much Udacity for providing such quality content. 

LDA is used to classify text in a document to a particular topic. It builds a topic per document model and words per topic model, modeled as Dirichlet distributions. 

* Each document is modeled as a multinomial distribution of topics and each topic is modeled as a multinomial distribution of words.
* LDA assumes that the every chunk of text we feed into it will contain words that are somehow related. Therefore choosing the right corpus of data is crucial. 
* It also assumes documents are produced from a mixture of topics. Those topics then generate words based on their probability distribution. 

## Load the dataset

The dataset we'll use is a list of over one million news headlines published over a period of 15 years. We'll start by loading it from the `abcnews-date-text.csv` file.

In [1]:
'''
Load the dataset from the csv and save it to 'data_text'
'''
import pandas as pd
data = pd.read_excel('output-merged.xlsx')
# we only need to headlines from the data
data_text = data[:300000][['Content']]
data_text['index'] = data_text.index
documents = data_text

Let's glance at the dataset:

In [2]:
'''
Get the total number of documents
'''
print(len(documents))

1249


In [3]:
documents.head()

,Content,index
0,"The waters of the Bay of Lakonikos, on the sou...",0
1,Public sector workers in an eastern Chinese ci...,1
2,Temu and Shein are taking off in the United St...,2
3,NBCUniversal CEO Jeff Shell is leaving the com...,3
4,America’s leading specialty baby products reta...,4


## Data Preprocessing ##

We will perform the following steps:

* **Tokenization**: Split the text into sentences and the sentences into words. Lowercase the words and remove punctuation.
* Words that have fewer than 3 characters are removed.
* All **stopwords** are removed.
* Words are **lemmatized** - words in third person are changed to first person and verbs in past and future tenses are changed into present.
* Words are **stemmed** - words are reduced to their root form.

In [4]:
'''
Loading Gensim and nltk libraries
'''

import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(400)

In [5]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to C:\Users\Yamini
[nltk_data]     Manral\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [6]:
'''
Write a function to perform the pre processing steps on the entire dataset
'''
stemmer = SnowballStemmer("english")
def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))

# Tokenize and Lemmatize
def preprocess(text):
    result=[]
    for token in gensim.utils.simple_preprocess(text) :
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result

In [10]:
'''
Preview a document after preprocessing
'''
document_num = 1248
doc_sample = documents[documents['index'] == document_num].values[0][0]

print("Original document: ")
words = []
for word in doc_sample.split(' '):
    words.append(word)
print(words)
print("\n\nTokenized and lemmatized document: ")
print(preprocess(doc_sample))

Original document: 
['Thomas', 'Fuchs,', 'the', 'Dean', 'of', 'Artificial', 'Intelligence', 'and', 'Human', 'Health', 'at', 'Mount', 'Sinai', 'in', 'NYC,', 'said', 'AI', 'will', 'be', 'needed', 'to', 'retain', 'the', 'standard', 'of', 'care', 'in', 'the', 'U.S.', 'ChatGPT,', 'the', 'artificial', 'intelligence', 'chatbot', 'that', 'was', 'released', 'by', 'OpenAI', 'in', 'December', '2022,', 'is', 'known', 'for', 'its', 'ability', 'to', 'answer', 'questions', 'and', 'provide', 'detailed', 'information', 'in', 'seconds', '—', 'all', 'in', 'a', 'clear,', 'conversational', 'way.', 'As', 'its', 'popularity', 'grows,', 'ChatGPT', 'is', 'popping', 'up', 'in', 'virtually', 'every', 'industry,', 'including', 'education,', 'real', 'estate,', 'content', 'creation', 'and', 'even', 'health', 'care.', 'Although', 'the', 'chatbot', 'could', 'potentially', 'change', 'or', 'improve', 'some', 'aspects', 'of', 'the', 'patient', 'experience,', 'experts', 'caution', 'that', 'it', 'has', 'limitations', 'and

['thoma', 'fuch', 'dean', 'artifici', 'intellig', 'human', 'health', 'mount', 'sinai', 'say', 'need', 'retain', 'standard', 'care', 'chatgpt', 'artifici', 'intellig', 'chatbot', 'releas', 'openai', 'decemb', 'know', 'abil', 'answer', 'question', 'provid', 'detail', 'inform', 'second', 'clear', 'convers', 'popular', 'grow', 'chatgpt', 'pop', 'virtual', 'industri', 'includ', 'educ', 'real', 'estat', 'content', 'creation', 'health', 'care', 'chatbot', 'potenti', 'chang', 'improv', 'aspect', 'patient', 'experi', 'expert', 'caution', 'limit', 'risk', 'substitut', 'physician', 'care', 'health', 'care', 'platform', 'predict', 'diabet', 'high', 'accuraci', 'replac', 'patient', 'care', 'search', 'medic', 'inform', 'onlin', 'peopl', 'googl', 'symptom', 'year', 'chatgpt', 'peopl', 'health', 'relat', 'question', 'engag', 'feel', 'like', 'interact', 'convers', 'seem', 'know', 'sourc', 'medic', 'inform', 'chatgpt', 'power', 'googl', 'certain', 'give', 'compel', 'result', 'result', 'right', 'wrong', 

<b>WAIT WHATTTTT Chat GPT? . That shows our accuracy in data </b>

<left><img src="images/sidelook.gif" width="200" height="100" /></left>

In [11]:
documents.head()

,Content,index
0,"The waters of the Bay of Lakonikos, on the sou...",0
1,Public sector workers in an eastern Chinese ci...,1
2,Temu and Shein are taking off in the United St...,2
3,NBCUniversal CEO Jeff Shell is leaving the com...,3
4,America’s leading specialty baby products reta...,4


Let's now preprocess all the news headlines we have. To do that, let's use the [map](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.Series.map.html) function from pandas to apply `preprocess()` to the `headline_text` column


In [13]:
# preprocess all the headlines, saving the list of results as 'processed_docs'
processed_docs = documents['Content'].map(preprocess)

In [14]:
'''
Preview 'processed_docs'
'''
processed_docs.head()

0    [water, lakoniko, south, eastern, greec, pelop...
1    [public, sector, worker, eastern, chines, citi...
2    [temu, shein, take, unit, state, top, store, c...
3    [nbcunivers, jeff, shell, leav, compani, outsi...
4    [america, lead, specialti, babi, product, reta...
Name: Content, dtype: object

## Bag of words on the dataset

Now let's create a dictionary from 'processed_docs' containing the number of times a word appears in the training set. To do that, let's pass `processed_docs` to [`gensim.corpora.Dictionary()`](https://radimrehurek.com/gensim/corpora/dictionary.html) and call it '`dictionary`'.

In [15]:
'''
Create a dictionary from 'processed_docs' containing the number of times a word appears 
in the training set using gensim.corpora.Dictionary and call it 'dictionary'
'''
dictionary = gensim.corpora.Dictionary(processed_docs)

In [16]:
'''
Checking dictionary created
'''
count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break

0 abl
1 accident
2 accord
3 account
4 affair
5 agenc
6 alarm
7 analyst
8 area
9 arriv
10 asia


** Gensim filter_extremes **
​
[`filter_extremes(no_below=5, no_above=0.5, keep_n=100000)`](https://radimrehurek.com/gensim/corpora/dictionary.html#gensim.corpora.dictionary.Dictionary.filter_extremes)
​
Filter out tokens that appear in
​
* less than no_below documents (absolute number) or
* more than no_above documents (fraction of total corpus size, not absolute number).
* after (1) and (2), keep only the first keep_n most frequent tokens (or keep all if None).

In [17]:
'''
Remove very rare and very common words:

- words appearing less than 15 times
- words appearing in more than 10% of all documents
'''
dictionary.filter_extremes(no_below=15, no_above=0.1, keep_n=100000)

** Gensim doc2bow **

[`doc2bow(document)`](https://radimrehurek.com/gensim/corpora/dictionary.html#gensim.corpora.dictionary.Dictionary.doc2bow)

* Convert document (a list of words) into the bag-of-words format = list of (token_id, token_count) 2-tuples. Each word is assumed to be a tokenized and normalized string (either unicode or utf8-encoded). No further preprocessing is done on the words in document; apply tokenization, stemming etc. before calling this method.

In [18]:
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]

In [19]:
'''
Checking Bag of Words corpus for our sample document --> (token_id, token_count)
'''
bow_corpus[document_num]

[(45, 1),
 (46, 2),
 (75, 1),
 (105, 5),
 (126, 1),
 (138, 2),
 (155, 1),
 (157, 1),
 (161, 1),
 (165, 1),
 (172, 2),
 (177, 1),
 (181, 1),
 (183, 2),
 (189, 1),
 (202, 1),
 (227, 1),
 (237, 1),
 (276, 1),
 (277, 1),
 (286, 5),
 (292, 2),
 (298, 1),
 (303, 1),
 (324, 1),
 (345, 2),
 (361, 1),
 (362, 1),
 (413, 1),
 (423, 1),
 (443, 1),
 (458, 1),
 (459, 1),
 (470, 1),
 (471, 1),
 (476, 1),
 (477, 1),
 (497, 1),
 (501, 1),
 (514, 4),
 (516, 1),
 (533, 2),
 (539, 1),
 (545, 2),
 (546, 1),
 (552, 21),
 (555, 1),
 (566, 2),
 (619, 3),
 (635, 1),
 (643, 1),
 (653, 1),
 (662, 1),
 (681, 1),
 (682, 2),
 (685, 1),
 (696, 1),
 (699, 1),
 (713, 1),
 (717, 1),
 (729, 1),
 (780, 2),
 (782, 2),
 (799, 3),
 (803, 1),
 (814, 1),
 (815, 1),
 (821, 2),
 (827, 1),
 (831, 2),
 (851, 2),
 (866, 2),
 (872, 1),
 (897, 1),
 (912, 1),
 (928, 18),
 (930, 5),
 (938, 1),
 (947, 1),
 (1005, 1),
 (1017, 1),
 (1049, 2),
 (1066, 1),
 (1091, 1),
 (1121, 1),
 (1124, 1),
 (1132, 2),
 (1156, 1),
 (1162, 1),
 (1283, 1),


In [20]:
'''
Preview BOW for our sample preprocessed document
'''
# Here document_num is document number 4310 which we have checked in Step 2
bow_doc_4310 = bow_corpus[document_num]

for i in range(len(bow_doc_4310)):
    print("Word {} (\"{}\") appears {} time.".format(bow_doc_4310[i][0], 
                                                     dictionary[bow_doc_4310[i][0]], 
                                                     bow_doc_4310[i][1]))

Word 45 ("instanc") appears 1 time.
Word 46 ("intellig") appears 2 time.
Word 75 ("practic") appears 1 time.
Word 105 ("system") appears 5 time.
Word 126 ("wrong") appears 1 time.
Word 138 ("custom") appears 2 time.
Word 155 ("lack") appears 1 time.
Word 157 ("mount") appears 1 time.
Word 161 ("onlin") appears 1 time.
Word 165 ("popular") appears 1 time.
Word 172 ("societi") appears 2 time.
Word 177 ("transit") appears 1 time.
Word 181 ("user") appears 1 time.
Word 183 ("virtual") appears 2 time.
Word 189 ("abil") appears 1 time.
Word 202 ("bloomberg") appears 1 time.
Word 227 ("decemb") appears 1 time.
Word 237 ("enabl") appears 1 time.
Word 276 ("lifestyl") appears 1 time.
Word 277 ("link") appears 1 time.
Word 286 ("model") appears 5 time.
Word 292 ("platform") appears 2 time.
Word 298 ("prove") appears 1 time.
Word 303 ("reli") appears 1 time.
Word 324 ("standard") appears 1 time.
Word 345 ("use") appears 2 time.
Word 361 ("content") appears 1 time.
Word 362 ("creation") appears 1 

## TF-IDF on our document set ##

While performing TF-IDF on the corpus is not necessary for LDA implemention using the gensim model, it is recemmended. TF-IDF expects a bag-of-words (integer values) training corpus during initialization. During transformation, it will take a vector and return another vector of the same dimensionality.

** TF-IDF stands for "Term Frequency, Inverse Document Frequency".**

* It is a way to score the importance of words (or "terms") in a document based on how frequently they appear across multiple documents.
* If a word appears frequently in a document, it's important. Give the word a high score. But if a word appears in many documents, it's not a unique identifier. Give the word a low score.
* Therefore, common words like "the" and "for", which appear in many documents, will be scaled down. Words that appear frequently in a single document will be scaled up.

In other words:

* TF(w) = `(Number of times term w appears in a document) / (Total number of terms in the document)`.
* IDF(w) = `log_e(Total number of documents / Number of documents with term w in it)`.

** For example **

* Consider a document containing `100` words wherein the word 'tiger' appears 3 times. 
* The term frequency (i.e., tf) for 'tiger' is then: 
    - `TF = (3 / 100) = 0.03`. 

* Now, assume we have `10 million` documents and the word 'tiger' appears in `1000` of these. Then, the inverse document frequency (i.e., idf) is calculated as:
    - `IDF = log(10,000,000 / 1,000) = 4`. 

* Thus, the Tf-idf weight is the product of these quantities: 
    - `TF-IDF = 0.03 * 4 = 0.12`.

In [21]:
 '''
Create tf-idf model object using models.TfidfModel on 'bow_corpus' and save it to 'tfidf'
'''
from gensim import corpora, models


tfidf = models.TfidfModel(bow_corpus)

In [22]:
'''
Apply transformation to the entire corpus and call it 'corpus_tfidf'
'''
corpus_tfidf = tfidf[bow_corpus]

In [23]:
'''
Preview TF-IDF scores for our first document --> --> (token_id, tfidf score)
'''
from pprint import pprint
for doc in corpus_tfidf:
    pprint(doc)
    break

[(0, 0.04125843474610488),
 (1, 0.029435204347037846),
 (2, 0.033078592726213975),
 (3, 0.028730451096232734),
 (4, 0.10545482152813564),
 (5, 0.03394846871435313),
 (6, 0.028595152819634796),
 (7, 0.029731308283632532),
 (8, 0.030192207688981478),
 (9, 0.024014905454625167),
 (10, 0.03118151612382772),
 (11, 0.15981053361891986),
 (12, 0.06533988682400711),
 (13, 0.16892806357791734),
 (14, 0.025998184007779854),
 (15, 0.08346726136718473),
 (16, 0.02642386520679574),
 (17, 0.028867565511255862),
 (18, 0.039952633404729966),
 (19, 0.03681171700206665),
 (20, 0.029435204347037846),
 (21, 0.03394846871435313),
 (22, 0.02418804299860464),
 (23, 0.041733630683592364),
 (24, 0.029290310136859104),
 (25, 0.02481823481948726),
 (26, 0.028730451096232734),
 (27, 0.026102952829238118),
 (28, 0.07959934860171172),
 (29, 0.026102952829238118),
 (30, 0.11865599669149493),
 (31, 0.0857854584589044),
 (32, 0.19045121152208117),
 (33, 0.035410225307101875),
 (34, 0.02418804299860464),
 (35, 0.204021

## Running LDA using Bag of Words ##

We are going for 10 topics in the document corpus.

** We will be running LDA using all CPU cores to parallelize and speed up model training.**

Some of the parameters we will be tweaking are:

* **num_topics** is the number of requested latent topics to be extracted from the training corpus.
* **id2word** is a mapping from word ids (integers) to words (strings). It is used to determine the vocabulary size, as well as for debugging and topic printing.
* **workers** is the number of extra processes to use for parallelization. Uses all available cores by default.
* **alpha** and **eta** are hyperparameters that affect sparsity of the document-topic (theta) and topic-word (lambda) distributions. We will let these be the default values for now(default value is `1/num_topics`)
    - Alpha is the per document topic distribution.
        * High alpha: Every document has a mixture of all topics(documents appear similar to each other).
        * Low alpha: Every document has a mixture of very few topics

    - Eta is the per topic word distribution.
        * High eta: Each topic has a mixture of most words(topics appear similar to each other).
        * Low eta: Each topic has a mixture of few words.

* ** passes ** is the number of training passes through the corpus. For  example, if the training corpus has 50,000 documents, chunksize is  10,000, passes is 2, then online training is done in 10 updates: 
    * `#1 documents 0-9,999 `
    * `#2 documents 10,000-19,999 `
    * `#3 documents 20,000-29,999 `
    * `#4 documents 30,000-39,999 `
    * `#5 documents 40,000-49,999 `
    * `#6 documents 0-9,999 `
    * `#7 documents 10,000-19,999 `
    * `#8 documents 20,000-29,999 `
    * `#9 documents 30,000-39,999 `
    * `#10 documents 40,000-49,999` 

In [45]:
# LDA mono-core -- fallback code in case LdaMulticore throws an error on your machine
# lda_model = gensim.models.LdaModel(bow_corpus, 
#                                    num_topics = 10, 
#                                    id2word = dictionary,                                    
#                                    passes = 50)

# LDA multicore 
'''
Train your lda model using gensim.models.LdaMulticore and save it to 'lda_model'
'''
lda_model = gensim.models.LdaMulticore(bow_corpus, 
                                       num_topics=5, 
                                       id2word = dictionary, 
                                       passes = 2, 
                                       workers=2)

In [46]:
'''
For each topic, we will explore the words occuring in that topic and its relative weight
'''
for idx, topic in lda_model.print_topics(-1):
    print("Topic: {} \nWords: {}".format(topic, idx ))
    print("\n")

Topic: 0.007*"rocket" + 0.006*"spacex" + 0.005*"black" + 0.005*"flight" + 0.005*"starship" + 0.004*"librari" + 0.003*"plastic" + 0.003*"hole" + 0.003*"king" + 0.003*"nasa" 
Words: 0


Topic: 0.012*"moon" + 0.010*"nasa" + 0.005*"orbit" + 0.005*"astronaut" + 0.004*"trump" + 0.004*"solar" + 0.003*"eclips" + 0.003*"crew" + 0.003*"republican" + 0.003*"spacecraft" 
Words: 1


Topic: 0.004*"plant" + 0.004*"pollut" + 0.003*"round" + 0.003*"goal" + 0.003*"leagu" + 0.003*"justic" + 0.003*"republican" + 0.003*"player" + 0.002*"hospit" + 0.002*"environment" 
Words: 2


Topic: 0.005*"anim" + 0.005*"drug" + 0.004*"trump" + 0.004*"republican" + 0.004*"student" + 0.003*"elect" + 0.003*"abort" + 0.003*"conserv" + 0.003*"sleep" + 0.003*"district" 
Words: 3


Topic: 0.006*"russia" + 0.006*"militari" + 0.005*"ukrain" + 0.004*"russian" + 0.004*"beij" + 0.004*"chines" + 0.003*"foreign" + 0.003*"nuclear" + 0.003*"minist" + 0.003*"store" 
Words: 4




In [37]:
import pyLDAvis
import pyLDAvis.gensim_models
import gensim.corpora as corpora
import gensim.models.ldamodel as lda

In [47]:
pyLDAvis.enable_notebook()
id2word=dictionary
vis = pyLDAvis.gensim_models.prepare(lda_model, bow_corpus, id2word)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2     -0.019910  0.032917       1        1  25.265356
4     -0.037281 -0.038985       2        1  22.532251
3     -0.017643  0.020378       3        1  21.377710
1      0.034669 -0.024786       4        1  16.036500
0      0.040164  0.010476       5        1  14.788183, topic_info=          Term        Freq       Total Category  logprob  loglift
2400      moon  524.000000  524.000000  Default  30.0000  30.0000
1891      nasa  470.000000  470.000000  Default  29.0000  29.0000
1346    rocket  315.000000  315.000000  Default  28.0000  28.0000
1915    spacex  236.000000  236.000000  Default  27.0000  27.0000
1916  starship  163.000000  163.000000  Default  26.0000  26.0000
...        ...         ...         ...      ...      ...      ...
1429      educ   59.737125  229.113982   Topic5  -6.1029   0.5671
2400      moon   73.796188  524.331989   Topic5  -5.8916  -0.0495
451    contain   56.475516  205.034895   Topic5  -6.1591   0.6220
65       ocean   55.218458  203.594651   Topic5  -6.1816   0.6065
973     minist   58.252818  296.491169   Topic5  -6.1281   0.2841

[313 rows x 6 columns], token_table=      Topic      Freq    Term
term                         
2408      1  0.070953  aboard
2408      2  0.070953  aboard
2408      3  0.141905  aboard
2408      4  0.591273  aboard
2408      5  0.118255  aboard
...     ...       ...     ...
2011      1  0.165259    wyom
2011      2  0.041315    wyom
2011      3  0.578407    wyom
2011      4  0.123944    wyom
2011      5  0.082630    wyom

[1295 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 5, 4, 2, 1])

## Running LDA using TF-IDF ##

In [43]:
'''
Define lda model using corpus_tfidf, again using gensim.models.LdaMulticore()
'''

lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, 
                                       num_topics=5, 
                                       id2word = dictionary, 
                                       passes = 2, 
                                       workers=2)

In [44]:
'''
For each topic, we will explore the words occuring in that topic and its relative weight
'''
for idx, topic in lda_model_tfidf.print_topics(-1):
    print("Topic: {} Word: {}".format(idx, topic))
    print("\n")

Topic: 0 Word: 0.003*"anim" + 0.003*"king" + 0.002*"driver" + 0.002*"abort" + 0.002*"austin" + 0.002*"round" + 0.002*"book" + 0.002*"charl" + 0.002*"black" + 0.002*"chemic"


Topic: 1 Word: 0.003*"isra" + 0.002*"golf" + 0.002*"sudan" + 0.002*"harri" + 0.002*"israel" + 0.002*"nuclear" + 0.002*"tenni" + 0.002*"militari" + 0.002*"driver" + 0.002*"ukrain"


Topic: 2 Word: 0.005*"moon" + 0.004*"nasa" + 0.003*"orbit" + 0.002*"rocket" + 0.002*"mar" + 0.002*"eclips" + 0.002*"telescop" + 0.002*"hunter" + 0.002*"astronaut" + 0.002*"solar"


Topic: 3 Word: 0.004*"leagu" + 0.003*"debt" + 0.003*"mccarthi" + 0.003*"pollut" + 0.003*"republican" + 0.002*"goal" + 0.002*"club" + 0.002*"asteroid" + 0.002*"champion" + 0.002*"electr"


Topic: 4 Word: 0.003*"russia" + 0.003*"russian" + 0.002*"ukrain" + 0.002*"speci" + 0.002*"musk" + 0.002*"spacex" + 0.002*"militari" + 0.002*"minist" + 0.002*"starship" + 0.002*"australian"




In [48]:
vis2 = pyLDAvis.gensim_models.prepare(lda_model_tfidf, corpus_tfidf, dictionary)
vis2

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4     -0.003256  0.019589       1        1  28.664021
0      0.011807 -0.012181       2        1  26.447860
2     -0.037274 -0.004459       3        1  20.721717
3      0.012704 -0.026759       4        1  14.407214
1      0.016018  0.023809       5        1   9.759188, topic_info=          Term       Freq      Total Category  logprob  loglift
2400      moon  13.000000  13.000000  Default  30.0000  30.0000
1838     leagu   8.000000   8.000000  Default  29.0000  29.0000
400       debt   5.000000   5.000000  Default  28.0000  28.0000
1785  mccarthi   5.000000   5.000000  Default  27.0000  27.0000
1655      isra   3.000000   3.000000  Default  26.0000  26.0000
...        ...        ...        ...      ...      ...      ...
251    foreign   1.162250   6.321647   Topic5  -6.5263   0.6333
928      medic   1.303788   8.687148   Topic5  -6.4114   0.4304
91     russian   1.327350   9.721233   Topic5  -6.3935   0.3358
90      russia   1.331902  10.165096   Topic5  -6.3901   0.2946
2123      hole   1.040030   7.235065   Topic5  -6.6374   0.3873

[284 rows x 6 columns], token_table=      Topic      Freq   Term
term                        
698       1  0.226750  abort
698       2  0.453500  abort
698       3  0.113375  abort
698       4  0.113375  abort
698       5  0.113375  abort
...     ...       ...    ...
2062      4  0.618549  wheel
2054      1  0.176800   wood
2054      2  0.530399   wood
2054      3  0.176800   wood
2054      5  0.176800   wood

[675 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[5, 1, 3, 4, 2])

## Performance evaluation by classifying sample document using LDA Bag of Words model

We will check to see where our test document would be classified.

In [49]:
'''
Text of sample document 1249
'''
processed_docs[1248]

['thoma',
 'fuch',
 'dean',
 'artifici',
 'intellig',
 'human',
 'health',
 'mount',
 'sinai',
 'say',
 'need',
 'retain',
 'standard',
 'care',
 'chatgpt',
 'artifici',
 'intellig',
 'chatbot',
 'releas',
 'openai',
 'decemb',
 'know',
 'abil',
 'answer',
 'question',
 'provid',
 'detail',
 'inform',
 'second',
 'clear',
 'convers',
 'popular',
 'grow',
 'chatgpt',
 'pop',
 'virtual',
 'industri',
 'includ',
 'educ',
 'real',
 'estat',
 'content',
 'creation',
 'health',
 'care',
 'chatbot',
 'potenti',
 'chang',
 'improv',
 'aspect',
 'patient',
 'experi',
 'expert',
 'caution',
 'limit',
 'risk',
 'substitut',
 'physician',
 'care',
 'health',
 'care',
 'platform',
 'predict',
 'diabet',
 'high',
 'accuraci',
 'replac',
 'patient',
 'care',
 'search',
 'medic',
 'inform',
 'onlin',
 'peopl',
 'googl',
 'symptom',
 'year',
 'chatgpt',
 'peopl',
 'health',
 'relat',
 'question',
 'engag',
 'feel',
 'like',
 'interact',
 'convers',
 'seem',
 'know',
 'sourc',
 'medic',
 'inform',
 'cha

In [50]:
'''
Check which topic our test document belongs to using the LDA Bag of Words model.
'''
document_num = 1248

# Our test document is document number 4310
for index, score in sorted(lda_model[bow_corpus[document_num]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model.print_topic(index, 10)))


Score: 0.9268770217895508	 
Topic: 0.006*"russia" + 0.006*"militari" + 0.005*"ukrain" + 0.004*"russian" + 0.004*"beij" + 0.004*"chines" + 0.003*"foreign" + 0.003*"nuclear" + 0.003*"minist" + 0.003*"store"

Score: 0.0657883957028389	 
Topic: 0.005*"anim" + 0.005*"drug" + 0.004*"trump" + 0.004*"republican" + 0.004*"student" + 0.003*"elect" + 0.003*"abort" + 0.003*"conserv" + 0.003*"sleep" + 0.003*"district"


### It has the highest probability (`0.92`) to be  part of the topic that we assigned as Topic X, which is the accurate classification. ###

## Performance evaluation by classifying sample document using LDA TF-IDF model

In [52]:
'''
Check which topic our test document belongs to using the LDA TF-IDF model.
'''
# Our test document is document number 4310
for index, score in sorted(lda_model_tfidf[bow_corpus[document_num]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model_tfidf.print_topic(index, 10)))


Score: 0.5435605049133301	 
Topic: 0.003*"anim" + 0.003*"king" + 0.002*"driver" + 0.002*"abort" + 0.002*"austin" + 0.002*"round" + 0.002*"book" + 0.002*"charl" + 0.002*"black" + 0.002*"chemic"

Score: 0.4540926218032837	 
Topic: 0.003*"russia" + 0.003*"russian" + 0.002*"ukrain" + 0.002*"speci" + 0.002*"musk" + 0.002*"spacex" + 0.002*"militari" + 0.002*"minist" + 0.002*"starship" + 0.002*"australian"


## It has the highest probability (54%) to be part of the topic that we assigned as topic X.

## Testing model on unseen document ##

In [53]:
unseen_document = "My favorite sports activities are running and swimming."

# Data preprocessing step for the unseen document
bow_vector = dictionary.doc2bow(preprocess(unseen_document))

for index, score in sorted(lda_model[bow_vector], key=lambda tup: -1*tup[1]):
    print("Score: {}\t Topic: {}".format(score, lda_model.print_topic(index, 5)))

Score: 0.7244266867637634	 Topic: 0.005*"anim" + 0.005*"drug" + 0.004*"trump" + 0.004*"republican" + 0.004*"student"
Score: 0.07061360776424408	 Topic: 0.004*"plant" + 0.004*"pollut" + 0.003*"round" + 0.003*"goal" + 0.003*"leagu"
Score: 0.0689956322312355	 Topic: 0.006*"russia" + 0.006*"militari" + 0.005*"ukrain" + 0.004*"russian" + 0.004*"beij"
Score: 0.06858508288860321	 Topic: 0.007*"rocket" + 0.006*"spacex" + 0.005*"black" + 0.005*"flight" + 0.005*"starship"
Score: 0.0673789456486702	 Topic: 0.012*"moon" + 0.010*"nasa" + 0.005*"orbit" + 0.005*"astronaut" + 0.004*"trump"


<left><img src="images/amazing.gif" width="200" height="100" /></left>

The model correctly classifies the unseen document with '72'% probability to the X category.

<left><img src="images/dua.gif" width="200" height="100" /></left>